## Imports

In [1]:
import os
import pandas as pd
import numpy as np

from deepface import DeepFace

## Creating dataframe, setting input location

In [2]:
assignment = pd.DataFrame(columns = ['filename', 'gender', 'race', 'age', 'emotion'])
input = 'faceimages'
images = os.listdir(input)

## Deepface Analysis, Filler Logic, DataFrame Appending

In [3]:
for img in images:
    predictions = DeepFace.analyze(img_path = os.path.join(input, img),
                            actions = ['gender', 'race', 'age', 'emotion'], 
                            enforce_detection=False)
    
    # Gender filler
    gender = predictions[0]['gender']
    gender_accepted = .80
    dominant_gender, gender_confidence = max(gender.items(), key = lambda item: item[1])
    if gender_confidence < gender_accepted:
        gender = 'Unknown'
    else:
        gender = dominant_gender

    # Race filler
    min_acc_confindent = 0.5
    race = predictions[0]['race']
    dominant_race, race_confidence = max(race.items(), key = lambda item: item[1])
    if race_confidence < min_acc_confindent:
        race = 'Unknown'
    else:
        race = dominant_race
    
    # Age filler
    age = predictions[0]['age']
    if np.isnan(age) or age == 0:
        age = -1

    # Not part of the assignment but I wanted to see how well this works
    emotion = predictions[0]['emotion']
    dominant_emotion, emotion_confidence = max(emotion.items(), key = lambda item: item[1])
    if emotion_confidence < min_acc_confindent:
        emotion = 'Unknown'
    else:
        emotion = dominant_emotion
    
    # Creating dictionary for adding a new row to the existing dataframe
    result_row = {'filename': img, 'gender': gender, 'race': race, 'age': age, 'emotion': emotion}

    # concatenating the new row to the dataframe 
    assignment = pd.concat([assignment, pd.DataFrame([result_row])], ignore_index=True)

Action: emotion: 100%|██████████| 4/4 [00:00<00:00,  6.04it/s]


## CSV Creation

In [4]:
assignment.to_csv('assignment.csv', index=False)